In [11]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [17]:
df = pd.read_csv("/home/mkapadni/work/crscore_plus_plus/claims_addressed_human_annotations.csv")
df.head()

,claim,review,diff,lang,label
0,The function call to 'unbound_dimensions' has ...,"The change seems fine, but it would be helpful...","@@ -204,7 +204,7 @@ class Renderer(Exporter):\...",py,1
1,"Now, an additional argument 'False' has been a...","The change seems fine, but it would be helpful...","@@ -204,7 +204,7 @@ class Renderer(Exporter):\...",py,1
2,The function call to 'unbound_dimensions' has ...,The unbound_dimensions function is not defined...,"@@ -204,7 +204,7 @@ class Renderer(Exporter):\...",py,1
3,"Now, an additional argument 'False' has been a...",I don't think you need `False` here.,"@@ -204,7 +204,7 @@ class Renderer(Exporter):\...",py,1
4,The function call to 'unbound_dimensions' has ...,The unbound_dimensions function should be call...,"@@ -204,7 +204,7 @@ class Renderer(Exporter):\...",py,1


In [18]:
df['lang'].unique()

array(['py', 'java', 'js'], dtype=object)

In [24]:


def extract_modified_code(diff_text):
    # Extract only the modified code (lines starting with '+')
    modified_lines = []
    in_code_block = False
    
    for line in diff_text.splitlines():
        if line.startswith('@@'):
            in_code_block = True
            continue
        if in_code_block and not line.startswith('-'):
            # Remove '+' if present, otherwise keep the context lines
            clean_line = line[1:] if line.startswith('+') else line
            modified_lines.append(clean_line)
    
    return '\n'.join(modified_lines)

def lint_code_changes(diff_text: str) -> str:
    # First extract the complete code block with context
    modified_code = extract_modified_code(diff_text)
    
    with tempfile.TemporaryDirectory() as tmp_dir:
        temp_file = Path(tmp_dir) / "code_to_check.py"
        # Add class definition to make the code valid Python
        full_code = "class Renderer(Exporter):\n    def method(self):\n" + modified_code
        temp_file.write_text(full_code)
        print(full_code)
        try:
            result = subprocess.run(
                [
                    "ruff", 
                    "check", 
                    str(temp_file),
                    "--format=json",
                    "--select=E,F,B,I,C,N,UP,YTT,S",
                ],
                capture_output=True,
                text=True
            )
            print(result)

            return result
        except subprocess.CalledProcessError as e:
            return f"Error running Ruff: {e.stderr}"

# Usage
diff_text = df['diff'][1]
results = lint_code_changes(diff_text)


class Renderer(Exporter):
    def method(self):
             if (((len(plot) == 1 and not plot.dynamic)
                 or (len(plot) > 1 and self.holomap is None) or
                 (plot.dynamic and len(plot.keys[0]) == 0)) or
                not unbound_dimensions(plot.streams, plot.dimensions, False)):
                 fmt = fig_formats[0] if self.fig=='auto' else self.fig
             else:
                 fmt = holomap_formats[0] if self.holomap=='auto' else self.holomap
CompletedProcess(args=['ruff', 'check', '/tmp/tmp96eo15od/code_to_check.py', '--format=json', '--select=E,F,B,I,C,N,UP,YTT,S'], returncode=2, stdout='', stderr="\x1b\x1berror:\x1b unexpected argument '\x1b--format\x1b' found\n\n  \x1btip:\x1b to pass '\x1b--format\x1b' as a value, use '\x1b-- --format\x1b'\n\n\x1b\x1bUsage:\x1b \x1bruff check\x1b <FILES|--fix|--no-fix|--unsafe-fixes|--no-unsafe-fixes|--show-fixes|--no-show-fixes|--diff|--watch|--fix-only|--no-fix-only|--ignore-noqa|--output-format <OUTPUT_FORM

In [28]:
print(results.stderr)

error: unexpected argument '--format' found

  tip: to pass '--format' as a value, use '-- --format'

Usage: ruff check <FILES|--fix|--no-fix|--unsafe-fixes|--no-unsafe-fixes|--show-fixes|--no-show-fixes|--diff|--watch|--fix-only|--no-fix-only|--ignore-noqa|--output-format <OUTPUT_FORMAT>|--output-file <OUTPUT_FILE>|--target-version <TARGET_VERSION>|--preview|--no-preview|--select <RULE_CODE>|--ignore <RULE_CODE>|--extend-select <RULE_CODE>|--extend-ignore <RULE_CODE>|--per-file-ignores <PER_FILE_IGNORES>|--extend-per-file-ignores <EXTEND_PER_FILE_IGNORES>|--exclude <FILE_PATTERN>|--extend-exclude <FILE_PATTERN>|--fixable <RULE_CODE>|--unfixable <RULE_CODE>|--extend-fixable <RULE_CODE>|--extend-unfixable <RULE_CODE>|--respect-gitignore|--no-respect-gitignore|--force-exclude|--no-force-exclude|--line-length <LINE_LENGTH>|--dummy-variable-rgx <DUMMY_VARIABLE_RGX>|--no-cache|--cache-dir <CACHE_DIR>|--stdin-filename <STDIN_FILENAME>|--extension <EXTENSION>|--exit-zero|--exit-non-zero-on-fi

In [29]:
print(results.stdout)

In [15]:
review_comments = lint_code_changes(df['diff'][1])
print("Ruff Review Results:")
print(review_comments)

Ruff Review Results:



In [2]:
df = pd.read_json("/home/mkapadni/work/crscore_plus_plus/Comment_Generation/msg-train_python_classified_temp.jsonl", lines=True)
df.head()

,oldf,patch,msg,id,y,raw_python_score,int_python_score
0,// | / |\n// ' / __| _` | _...,"@@ -595,8 +595,10 @@ namespace Kratos\n ...",I assumed that for CrossProduct the values wer...,90017,1,1.924489,2
1,#\n# Licensed to the Apache Software Foundatio...,"@@ -22,8 +22,13 @@\n For internal use only; no...",I think we should we avoid `import six` for co...,144914,1,1.455082,1
2,# frozen_string_literal: true\n\nrequire 'view...,"@@ -25,13 +25,16 @@ module View\n def sh...","we call cities + towns . size a lot, maybe mak...",12959,1,2.106942,2
3,/*\n * (C) Copyright 2016-2021 Intel Corporati...,"@@ -947,6 +947,7 @@ out:\n \t\tD_ERROR(""pool ""...",This will be removed.,171235,1,1.315966,1
4,// Licensed to Elasticsearch B.V. under one or...,"@@ -160,6 +160,11 @@ func (r *routeBuilder) pr...",nit: `firehoseLogHandler` vs. `firehoseMiddlew...,32732,1,1.084271,1


In [3]:
df.shape

(100, 7)

In [14]:
df_val = pd.read_json("/home/mkapadni/work/crscore_plus_plus/Comment_Generation/msg-valid.jsonl", lines=True)
df_val.head()

,patch,y,oldf,idx,id,msg,proj,lang
0,"@@ -231,4 +231,8 @@ def setup_app(app):\n ...",1,# -*- coding: utf-8 -*-\n#\n# This file is par...,1,16014,Should we call it `is_list`?,inveniosoftware-invenio,py
1,"@@ -44,7 +44,7 @@ namespace OpenTelemetry.Trac...",1,"// <copyright file=""TracerProviderBuilderExten...",1,18299,"in the instrumentation example, should we use ...",open-telemetry-opentelemetry-dotnet,.cs
2,"@@ -25,7 +25,7 @@ from scapy.modules.six.moves...",1,## This file is part of Scapy\n## See http://w...,1,12313,Why this change ? Is it useful ?,secdev-scapy,py
3,"@@ -0,0 +1,4 @@\n+const titleNode = virtualNod...",1,,1,15216,"I know this is a nitpick, but don't we always ...",dequelabs-axe-core,js
4,"@@ -37,6 +37,11 @@ public class EMailValidator...",1,package edu.harvard.iq.dataverse;\n\nimport st...,1,37751,We should reformat this emails in the test to ...,IQSS-dataverse,java


In [15]:
df_val.columns

Index(['patch', 'y', 'oldf', 'idx', 'id', 'msg', 'proj', 'lang'], dtype='object')

In [33]:
df_val['lang'].unique()

array(['py', '.cs', 'js', 'java', 'rb', 'php', 'go', 'c', 'cpp'],
      dtype=object)

# concatenating all dfs

In [53]:
df = pd.read_json("/home/mkapadni/work/crscore_plus_plus/Comment_Generation/msg-train_all_classified.json")
df.head()

,oldf,patch,msg,id,y,raw_score,int_score,lang,is_valid_syntax
0,,"@@ -0,0 +1,18 @@\n+package com.baeldung.lcm;\n...",Observation: absNumber1 and absNumber2 do not ...,82345,1,2.549807,3,c,NaN
1,/**\n * @author omkarlanghe\n * Given an array...,"@@ -31,12 +31,13 @@\n * @source: https://prac...",As you have added <bits/stdc++.h> you don't ne...,25195,1,2.696151,3,c,NaN
2,"""""""\nVarious utilities that don't fit anwhere ...","@@ -49,6 +49,8 @@ def sanitize(x: Any) -> Any:...",Would `elif x is None` work?,28616,1,2.557044,3,go,NaN
3,"// Copyright 2016 VMware, Inc. All Rights Rese...","@@ -57,15 +57,8 @@ func NewBackoffConfig() *Ba...",Thanks for this change :) I should have made t...,36092,1,2.510409,3,go,NaN
4,"package main\n\nimport (\n\t""fmt""\n\t""net""\n\t...","@@ -747,10 +747,16 @@ func logGRPCServerSetup(...",So this means that if multiple things error th...,42369,1,2.581007,3,go,NaN


In [54]:
df['lang'].value_counts()

lang
java      32756
python    25416
js         4291
php        1720
cpp         260
ruby        108
go           26
c             2
Name: count, dtype: int64

In [55]:
df_deduplicated = df.drop_duplicates(subset=['oldf', 'patch', 'msg'], keep='first')
df_deduplicated.shape

(56031, 9)

In [57]:
df_deduplicated['lang'].value_counts()

lang
java      30797
python    20888
js         2542
php        1641
cpp         114
ruby         25
go           22
c             2
Name: count, dtype: int64

In [61]:
df_train = pd.read_json("/home/mkapadni/work/crscore_plus_plus/Comment_Generation/msg-train.jsonl", lines=True)
df_train_deduplicated = df_train.drop_duplicates(subset=['oldf', 'patch', 'msg'], keep='first')
df_train_deduplicated.shape

(109274, 5)

In [63]:
df_train_deduplicated['lang'] = 'unk'

/tmp/ipykernel_1820976/80913064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_deduplicated['lang'] = 'unk'


In [64]:
df_train_deduplicated.head()

,oldf,patch,msg,id,y,lang
0,// | / |\n// ' / __| _` | _...,"@@ -595,8 +595,10 @@ namespace Kratos\n ...",I assumed that for CrossProduct the values wer...,90017,1,unk
1,#\n# Licensed to the Apache Software Foundatio...,"@@ -22,8 +22,13 @@\n For internal use only; no...",I think we should we avoid `import six` for co...,144914,1,unk
2,# frozen_string_literal: true\n\nrequire 'view...,"@@ -25,13 +25,16 @@ module View\n def sh...","we call cities + towns . size a lot, maybe mak...",12959,1,unk
3,/*\n * (C) Copyright 2016-2021 Intel Corporati...,"@@ -947,6 +947,7 @@ out:\n \t\tD_ERROR(""pool ""...",This will be removed.,171235,1,unk
4,// Licensed to Elasticsearch B.V. under one or...,"@@ -160,6 +160,11 @@ func (r *routeBuilder) pr...",nit: `firehoseLogHandler` vs. `firehoseMiddlew...,32732,1,unk


In [65]:
df_deduplicated.shape

(56031, 9)

In [66]:
# add those rows from df_train_deduplicated to df_deduplicated that are not already present in df_deduplicated
df_deduplicated = pd.concat([df_deduplicated, df_train_deduplicated]).drop_duplicates(subset=['oldf', 'patch', 'msg'], keep='first')
df_deduplicated.shape

(109274, 9)

In [67]:
df_deduplicated['lang'].value_counts()

lang
unk       53243
java      30797
python    20888
js         2542
php        1641
cpp         114
ruby         25
go           22
c             2
Name: count, dtype: int64

In [69]:
df_deduplicated.to_json("/home/mkapadni/work/crscore_plus_plus/Comment_Generation/msg-train_all_classified.json", orient='records', lines=True)